版权所有 2025 Google LLC。

In [ ]:
#@title 根据 Apache 2.0 许可证授权；
# 除非遵守许可证，否则不得使用此文件。
# 您可从以下地址获取许可证副本：
#
# https://www.apache.org/licenses/LICENSE-2.0 
#
# 除非适用法律要求或书面同意，否则按“原样”分发软件，
# 不附带任何明示或暗示的保证或条件。
# 详见许可证中关于权限与限制的具体条款。

<table align="left">
  <td>
      <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Convert_Gemma_3_270M_to_ONNX.ipynb "><img src="https://www.tensorflow.org/images/colab_logo_32px.png " />在 Google Colab 运行</a>
  </td>
</table>

# 将 Gemma 3 270M 转换为 ONNX

本笔记本将 Gemma 3 模型导出为 ONNX 格式，以便在 [Transformers.js](https://huggingface.co/docs/transformers.js/en/index ) 中使用，后者借助 ONNX Runtime 在浏览器里运行模型。全程不到 10 分钟：

1. 配置 Colab 环境
2. 从 Hugging Face 加载模型
3. 使用 Optimum 转换脚本导出模型
4. 测试、评估并保存模型以供后续使用

Gemma 3 270M 专为特定任务微调而设计，针对移动、网页和边缘设备的高效性能进行了优化。您可以在此[笔记本](https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Fine_tune_Gemma_3_270M_for_emoji_generation.ipynb )中微调自己的模型，转换后即可在[网页演示](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-transformersjs )中运行。

## 搭建开发环境

第一步是用 pip 安装依赖包。

In [ ]:
%pip install transformers==4.56.1 onnx==1.19.0 onnx_ir==0.1.7 onnxruntime==1.22.1 numpy==2.3.2 huggingface_hub

重启会话运行时，以确保使用刚安装的新版本包。

## 转换模型
若要访问并保存模型到 Hugging Face，请提供您的[访问令牌](https://huggingface.co/settings/tokens )。可将其设为环境变量,名称填 `HF_TOKEN`，值填您的专属令牌。

In [ ]:
import os
from huggingface_hub import login
login(hf_token)

接下来运行 [Xenova](https://huggingface.co/Xenova ) 提供的 build_gemma.py 脚本，对 Gemma 3 模型进行转换与量化。

指定待转换模型的 Hugging Face 仓库 ID，导出的 .onnx 文件将保存到您的 Colab 文件目录。

In [ ]:
!wget https://gist.githubusercontent.com/xenova/a219dbf3c7da7edd5dbb05f92410d7bd/raw/45f4c5a5227c1123efebe1e36d060672ee685a8e/build_gemma.py 

model_author = ""                                         #@param {type:"string"}
gemma_model = "myemoji-gemma-3-270m-it"                   #@param {type:"string"}

repo_id   = f"{model_author}/{gemma_model}"               # 要转换的模型
save_path = f"/content/{gemma_model}-onnx"                # 保存 resized 模型的路径

!python build_gemma.py \
    --model_name {repo_id} \
    --output {save_path} \
    -p fp32 fp16 q4 q4f16

print(f"已转换的 ONNX 模型保存在 {save_path}")

## 测试转换后的模型

当 .onnx 模型已保存到 Colab 会话后，可用 ONNX Runtime 测试推理。注意此处用的 ONNX Runtime 与浏览器中运行的 ONNX Runtime Web 略有差异。

可在 `text_to_translate` 中尝试不同文本，并对比各量化版本的性能。

In [ ]:
from transformers import AutoConfig, AutoTokenizer, GenerationConfig
import onnxruntime
import numpy as np

# 加载配置、分词器与模型
config = AutoConfig.from_pretrained(save_path)
generation_config = GenerationConfig.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

model_file = "onnx/model.onnx"          #@param ["onnx/model.onnx", "onnx/model_fp16.onnx", "onnx/model_q4.onnx", "onnx/model_q4f16.onnx"]

model_path = f"{save_path}/{model_file}"
decoder_session = onnxruntime.InferenceSession(model_path)

## 设置配置值
num_key_value_heads = config.num_key_value_heads
head_dim = config.head_dim
num_hidden_layers = config.num_hidden_layers
eos_token_id = tokenizer.eos_token_id

# 准备输入
text_to_translate = "i love sushi"      # @param {type:"string"}
messages = [
  { "role": "system", "content": "将这段文本翻译成表情符号：" },
  { "role": "user", "content": text_to_translate },
]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="np")
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
batch_size = input_ids.shape[0]
past_key_values = {
    f'past_key_values.{layer}.{kv}': np.zeros([batch_size, num_key_value_heads, 0, head_dim], dtype=np.float32)
    for layer in range(num_hidden_layers)
    for kv in ('key', 'value')
}
position_ids = np.tile(np.arange(0, input_ids.shape[-1]), (batch_size, 1))

# 3. 生成循环
max_new_tokens = 8
generated_tokens = np.array([[]], dtype=np.int64)

for i in range(max_new_tokens):
  logits, *present_key_values = decoder_session.run(None, dict(
      input_ids=input_ids,
      attention_mask=attention_mask,
      position_ids=position_ids,
      **past_key_values,
  ))

  ## 更新下一轮生成所需的值
  input_ids = logits[:, -1].argmax(-1, keepdims=True)
  attention_mask = np.concatenate([attention_mask, np.ones_like(input_ids, dtype=np.int64)], axis=-1)
  position_ids = position_ids[:, -1:] + 1

  for j, key in enumerate(past_key_values):
    past_key_values[key] = present_key_values[j]

  generated_tokens = np.concatenate([generated_tokens, input_ids], axis=-1)

  if np.isin(input_ids, eos_token_id).any():
    break

# 4. 输出结果
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

## 上传到 Hugging Face Hub

将导出的 ONNX 模型上传到 Hugging Face，方便分享与使用。

In [ ]:
import huggingface_hub
from huggingface_hub import whoami

username = whoami()['name']

#@markdown 给您的 ONNX 模型起个名字：
onnx_model = "myemoji-gemma-3-270m-it-onnx"       #@param {type:"string"}
hf_repo_id = f"{username}/{onnx_model}"

huggingface_hub.create_repo(hf_repo_id, exist_ok=True)

repo_url = huggingface_hub.upload_folder(
  folder_path=save_path,
  repo_id=hf_repo_id,
  repo_type="model",
  commit_message=f"上传 {onnx_model} 的 ONNX 模型文件"
  )

print(f"已上传至 {repo_url}")

## 使用 Transformers.js 部署到网页

现在您可以通过 ONNX Runtime Web，借助 [Transformers.js](https://huggingface.co/docs/transformers.js/en/index ) 在浏览器中运行 Gemma 3 模型。可在[表情符号生成网页演示](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-transformersjs )中体验，模型完全在浏览器端运行。